In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import sklearn
from sklearn.preprocessing import MinMaxScaler
import pandas as pd
from collections import defaultdict
from utils import extract_and_prepare_data, detrend_ue, detrend_ue_w_quality
from pathlib import Path
import pathlib
import argparse

normalize = True

methods_dict = {
    'MaximumSequenceProbability': 'MSP',
    'Perplexity': 'PPL',
    'MeanTokenEntropy': 'MTE',
    'MonteCarloSequenceEntropy': 'MCSE',
    'MonteCarloNormalizedSequenceEntropy': 'MCNSE',
    'LexicalSimilarity_rougeL': 'LSRL',
}

MODELS = {
    'llama': 'Llama 3.1 8B',
    'gemma': 'Gemma 2 9B',
    'eurollm': 'EuroLLM 9B',
}

DATASETS = {
    'metricx-metricx-24-hybrid-xxl-v2p6': [
        'wmt14_csen',
        'wmt14_deen',
        'wmt14_ruen',
        'wmt14_fren',
        'wmt19_deen',
        'wmt19_fien',
        'wmt19_lten',
        'wmt19_ruen',
    ],
    'XComet-XCOMET-XXL': [
        'wmt14_csen',
        'wmt14_deen',
        'wmt14_ruen',
        'wmt14_fren',
        'wmt19_deen',
        'wmt19_fien',
        'wmt19_lten',
        'wmt19_ruen',
    ],
    'Comet-wmt22-comet-da': [
        'wmt14_csen',
        'wmt14_deen',
        'wmt14_ruen',
        'wmt14_fren',
        'wmt19_deen',
        'wmt19_fien',
        'wmt19_lten',
        'wmt19_ruen',
    ],
    'bleu_proper': [
        'wmt14_csen',
        'wmt14_deen',
        'wmt14_ruen',
        'wmt14_fren',
        'wmt19_deen',
        'wmt19_fien',
        'wmt19_lten',
        'wmt19_ruen',
    ],
}

METRICS = {
    'metricx-metricx-24-hybrid-xxl-v2p6': 'MetricX XXL',
    'XComet-XCOMET-XXL': 'XComet XXL',
    'Comet-wmt22-comet-da': 'Comet WMT22',
    # 'bleu_proper': 'BLEU',
}

pathlib.Path('tables').mkdir(parents=True, exist_ok=True)
pathlib.Path('charts').mkdir(parents=True, exist_ok=True)

def get_header(caption):
    return (
        "\\begin{table*}\n"
        "\\footnotesize\n"
        "\\centering\n"
        f"\caption{{{caption}}}\n"
        "\\begin{tabular}{lcccccccc}\n"
        "\\toprule\n"
        "&\multicolumn{4}{c}{\\textbf{WMT14}}&\multicolumn{4}{c}{\\textbf{WMT19}}\\\\\n"
        "\cmidrule(lr){2-5}\n"
        "\cmidrule(lr){6-9}\n"

    )

def footer():
    return (
        "\midrule\n"
        "\end{tabular}\n"
        "\end{table*}\n"
    )

def colname(dataset):
    if '_' in dataset:
        dataset = dataset.split('_')[1]

    return dataset[:2].capitalize() + '-' + dataset[2:].capitalize()


def main(type_):
    for metric, metric_name in METRICS.items():
        datasets = DATASETS[metric]
        ue_methods = list(methods_dict.values())

        caption = f"Detailed PRR scores (raw and detrended) for all methods. Metric: {metric_name}."
        header = get_header(caption)
        header += "&" + "&".join([colname(dataset).replace('_', '\\_') for dataset in datasets]) + "\\\\\n"
        latex = header

        for model_type in ['base']:
            for model, model_name in MODELS.items():
                model_title = model_name if model_type == 'base' else f"{model_name} Instruct"
                latex += "\midrule\n"
                latex += "& \\multicolumn{8}{c}{" + model_title + "}\\\\\n"
                latex += "\midrule\n"

                ue_scores, _, _ = detrend_ue(datasets, model, model_type, [metric], ue_methods, methods_dict, return_unprocessed=True)

                # Find best and second-best scores per dataset (column-wise)
                best_map = {}
                second_map = {}
                for col_idx in range(len(datasets)):
                    scores = []
                    for method_short in methods_dict.values():
                        raw = ue_scores[f"{method_short}_raw"][col_idx]
                        detr = ue_scores[f"{method_short}_detr"][col_idx]
                        scores.extend([raw, detr])
                    sorted_scores = sorted(scores, reverse=True)
                    best_map[col_idx] = sorted_scores[0]
                    second_map[col_idx] = sorted_scores[1]

                for method_full, method_short in methods_dict.items():
                    raw_scores = ue_scores[f"{method_short}_raw"]
                    detr_scores = ue_scores[f"{method_short}_detr"]

                    raw_row = [f"Raw {method_short}"]
                    detr_row = [f"Detr {method_short}"]

                    for col_idx, (raw, detr) in enumerate(zip(raw_scores, detr_scores)):
                        # Format raw
                        if np.isclose(raw, best_map[col_idx]):
                            raw_row.append(f"\\textbf{{{raw:.2f}}}")
                        elif np.isclose(raw, second_map[col_idx]):
                            raw_row.append(f"\\underline{{{raw:.2f}}}")
                        else:
                            raw_row.append(f"{raw:.2f}")

                        # Format detr
                        if np.isclose(detr, best_map[col_idx]):
                            detr_row.append(f"\\textbf{{{detr:.2f}}}")
                        elif np.isclose(detr, second_map[col_idx]):
                            detr_row.append(f"\\underline{{{detr:.2f}}}")
                        else:
                            detr_row.append(f"{detr:.2f}")

                    latex += " & ".join(raw_row) + " \\\\\n"
                    latex += " & ".join(detr_row) + " \\\\\n"
                    latex += "\\midrule\n"

        latex += footer()
        name = f'tables/{metric}_detailed_results_per_dataset_ranking.tex'
        with open(name, 'w') as f:
            f.write(latex)


if __name__ == "__main__":
    type_='all'
    main(type_)

/home/maiya.goloburda/.conda/envs/detrend/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/home/maiya.goloburda/.conda/envs/detrend/lib/python3.10/site-packages/torch/cuda/__init__.py:734: UserWarning: Can't initialize NVML
  warnings.warn("Can't initialize NVML")
/home/maiya.goloburda/.conda/envs/detrend/lib/python3.10/site-packages/torch/cuda/__init__.py:734: UserWarning: Can't initialize NVML
  warnings.warn("Can't initialize NVML")
Exception ignored in: <bound method IPythonKernel._clean_thread_parent_frames of <ipykernel.ipkernel.IPythonKernel object at 0x7f52718e0e50>>
Traceback (most recent call last):
  File "/home/maiya.goloburda/.conda/envs/detrend/lib/python3.10/site-packages/ipykernel/ipkernel.py", line 775, in _clean_thread_parent_frames
    def _clean_thread_parent_frames(
KeyboardInterrup

Stat calculators: [<lm_polygraph.stat_calculators.greedy_probs.GreedyProbsCalculator object at 0x7f5024f17ac0>]


Some weights of the model checkpoint at microsoft/deberta-large-mnli were not used when initializing DebertaForSequenceClassification: ['config']
- This IS expected if you are initializing DebertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DebertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Stat calculators: [<lm_polygraph.stat_calculators.greedy_probs.GreedyProbsCalculator object at 0x7f5024f017b0>]
llama wmt14_csen Below q ids: 1771


Error during conversion: ChunkedEncodingError(ProtocolError('Response ended prematurely'))
/home/maiya.goloburda/.conda/envs/detrend/lib/python3.10/site-packages/torch/cuda/__init__.py:734: UserWarning: Can't initialize NVML
  warnings.warn("Can't initialize NVML")
Some weights of the model checkpoint at microsoft/deberta-large-mnli were not used when initializing DebertaForSequenceClassification: ['config']
- This IS expected if you are initializing DebertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DebertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Stat calculators: [<lm_polygraph.stat_calculators.greedy_probs.GreedyProbsCalculator object at 0x7f5024f48e80>]


Some weights of the model checkpoint at microsoft/deberta-large-mnli were not used when initializing DebertaForSequenceClassification: ['config']
- This IS expected if you are initializing DebertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DebertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Stat calculators: [<lm_polygraph.stat_calculators.greedy_probs.GreedyProbsCalculator object at 0x7f4fd0182ef0>]
llama wmt14_deen Below q ids: 1796


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import sklearn
from sklearn.preprocessing import MinMaxScaler
import pandas as pd
from collections import defaultdict
from utils import extract_and_prepare_data, detrend_ue, detrend_ue_w_quality
from pathlib import Path
import pathlib
import argparse

normalize = True

methods_dict = {
    'MaximumSequenceProbability': 'MSP',
    'Perplexity': 'PPL',
    'MeanTokenEntropy': 'MTE',
    'MonteCarloSequenceEntropy': 'MCSE',
    'MonteCarloNormalizedSequenceEntropy': 'MCNSE',
    'LexicalSimilarity_rougeL': 'LSRL',
}

MODELS = {
    'llama': 'Llama 3.1 8B',
    'gemma': 'Gemma 2 9B',
}

DATASETS = {
    'metricx-metricx-24-hybrid-xxl-v2p6': [
        'wmt14_csen',
        'wmt14_deen',
        'wmt14_ruen',
        'wmt14_fren',
        'wmt19_deen',
        'wmt19_fien',
        'wmt19_lten',
        'wmt19_ruen',
    ],
    'XComet-XCOMET-XXL': [
        'wmt14_csen',
        'wmt14_deen',
        'wmt14_ruen',
        'wmt14_fren',
        'wmt19_deen',
        'wmt19_fien',
        'wmt19_lten',
        'wmt19_ruen',
    ],
    'Comet-wmt22-comet-da': [
        'wmt14_csen',
        'wmt14_deen',
        'wmt14_ruen',
        'wmt14_fren',
        'wmt19_deen',
        'wmt19_fien',
        'wmt19_lten',
        'wmt19_ruen',
    ],
    'bleu_proper': [
        'wmt14_csen',
        'wmt14_deen',
        'wmt14_ruen',
        'wmt14_fren',
        'wmt19_deen',
        'wmt19_fien',
        'wmt19_lten',
        'wmt19_ruen',
    ],
    'Accuracy':['gsm8k'],
    'AlignScoreInputOutput':['xsum']
}

METRICS = {
    'Accuracy': 'Accuracy',
    'AlignScoreInputOutput': 'Align Score',
    # 'Comet-wmt22-comet-da': 'Comet WMT22',
    # 'bleu_proper': 'BLEU',
}

pathlib.Path('tables').mkdir(parents=True, exist_ok=True)
pathlib.Path('charts').mkdir(parents=True, exist_ok=True)

def get_header(caption):
    return (
        "\\begin{table*}\n"
        "\\footnotesize\n"
        "\\centering\n"
        f"\caption{{{caption}}}\n"
        "\\begin{tabular}{lc}\n"
                "\\toprule\n"
    )

def footer():
    return (
        "\midrule\n"
        "\end{tabular}\n"
        "\end{table*}\n"
    )

def colname(dataset):
    if '_' in dataset:
        dataset = dataset.split('_')[1]

    return dataset[:2].capitalize() + '-' + dataset[2:].capitalize()


def main(type_):
    for metric, metric_name in METRICS.items():
        datasets = DATASETS[metric]
        ue_methods = list(methods_dict.values())

        caption = f"Detailed PRR scores (raw and detrended) for all methods. Metric: {metric_name}."
        header = get_header(caption)
        header += "&" + "&".join([colname(dataset).replace('_', '\\_') for dataset in datasets]) + "\\\\\n"
        latex = header

        for model_type in ['base']:
            for model, model_name in MODELS.items():
                model_title = model_name if model_type == 'base' else f"{model_name} Instruct"
                latex += "\midrule\n"
                latex += "& \\multicolumn{8}{c}{" + model_title + "}\\\\\n"
                latex += "\midrule\n"

                ue_scores, _ , _, _,_= detrend_ue_w_quality(datasets, model, model_type, [metric], ue_methods, methods_dict, return_unprocessed=True)

                # Find best and second-best scores per dataset (column-wise)
                best_map = {}
                second_map = {}
                for col_idx in range(len(datasets)):
                    scores = []
                    for method_short in methods_dict.values():
                        raw = ue_scores[f"{method_short}_raw"][col_idx]
                        detr = ue_scores[f"{method_short}_detr"][col_idx]
                        scores.extend([raw, detr])
                    sorted_scores = sorted(scores, reverse=True)
                    best_map[col_idx] = sorted_scores[0]
                    second_map[col_idx] = sorted_scores[1]

                for method_full, method_short in methods_dict.items():
                    raw_scores = ue_scores[f"{method_short}_raw"]
                    detr_scores = ue_scores[f"{method_short}_detr"]

                    raw_row = [f"Raw {method_short}"]
                    detr_row = [f"Detr {method_short}"]

                    for col_idx, (raw, detr) in enumerate(zip(raw_scores, detr_scores)):
                        # Format raw
                        if np.isclose(raw, best_map[col_idx]):
                            raw_row.append(f"\\textbf{{{raw:.2f}}}")
                        elif np.isclose(raw, second_map[col_idx]):
                            raw_row.append(f"\\underline{{{raw:.2f}}}")
                        else:
                            raw_row.append(f"{raw:.2f}")

                        # Format detr
                        if np.isclose(detr, best_map[col_idx]):
                            detr_row.append(f"\\textbf{{{detr:.2f}}}")
                        elif np.isclose(detr, second_map[col_idx]):
                            detr_row.append(f"\\underline{{{detr:.2f}}}")
                        else:
                            detr_row.append(f"{detr:.2f}")

                    latex += " & ".join(raw_row) + " \\\\\n"
                    latex += " & ".join(detr_row) + " \\\\\n"
                    latex += "\\midrule\n"

        latex += footer()
        name = f'tables/{metric}_detailed_results_per_dataset_ranking_w_quality.tex'
        with open(name, 'w') as f:
            f.write(latex)


if __name__ == "__main__":
    type_='all'
    main(type_)

/home/maiya.goloburda/.conda/envs/detrend/lib/python3.10/site-packages/torch/cuda/__init__.py:734: UserWarning: Can't initialize NVML
  warnings.warn("Can't initialize NVML")
Some weights of the model checkpoint at microsoft/deberta-large-mnli were not used when initializing DebertaForSequenceClassification: ['config']
- This IS expected if you are initializing DebertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DebertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Stat calculators: [<lm_polygraph.stat_calculators.greedy_probs.GreedyProbsCalculator object at 0x7f44b82484c0>]


Error during conversion: ChunkedEncodingError(ProtocolError('Response ended prematurely'))
Some weights of the model checkpoint at microsoft/deberta-large-mnli were not used when initializing DebertaForSequenceClassification: ['config']
- This IS expected if you are initializing DebertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DebertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Stat calculators: [<lm_polygraph.stat_calculators.greedy_probs.GreedyProbsCalculator object at 0x7f44b842f970>]
llama gsm8k Below q ids: 1796


Error during conversion: ChunkedEncodingError(ProtocolError('Response ended prematurely'))
/home/maiya.goloburda/.conda/envs/detrend/lib/python3.10/site-packages/torch/cuda/__init__.py:734: UserWarning: Can't initialize NVML
  warnings.warn("Can't initialize NVML")
Some weights of the model checkpoint at microsoft/deberta-large-mnli were not used when initializing DebertaForSequenceClassification: ['config']
- This IS expected if you are initializing DebertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DebertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


...

Stat calculators: [<lm_polygraph.stat_calculators.greedy_probs.GreedyProbsCalculator object at 0x7f44b842f7f0>]
llama xsum Below q ids: 1620


/home/maiya.goloburda/.conda/envs/detrend/lib/python3.10/site-packages/torch/cuda/__init__.py:734: UserWarning: Can't initialize NVML
  warnings.warn("Can't initialize NVML")
Some weights of the model checkpoint at microsoft/deberta-large-mnli were not used when initializing DebertaForSequenceClassification: ['config']
- This IS expected if you are initializing DebertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DebertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Stat calculators: [<lm_polygraph.stat_calculators.greedy_probs.GreedyProbsCalculator object at 0x7f44b842d360>]


Some weights of the model checkpoint at microsoft/deberta-large-mnli were not used when initializing DebertaForSequenceClassification: ['config']
- This IS expected if you are initializing DebertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DebertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Stat calculators: [<lm_polygraph.stat_calculators.greedy_probs.GreedyProbsCalculator object at 0x7f426e61a470>]
gemma xsum Below q ids: 1781


In [3]:
import numpy as np
import matplotlib.pyplot as plt
import sklearn
from sklearn.preprocessing import MinMaxScaler
import pandas as pd
from collections import defaultdict
from utils import extract_and_prepare_data, detrend_ue
from pathlib import Path
import pathlib
import argparse

normalize = True

methods_dict = {
    'MaximumSequenceProbability': 'MSP',
    'Perplexity': 'PPL',
    'MeanTokenEntropy': 'MTE',
    'MonteCarloSequenceEntropy': 'MCSE',
    'MonteCarloNormalizedSequenceEntropy': 'MCNSE',
    'LexicalSimilarity_rougeL': 'LSRL',
}

MODELS = {
    'llama': 'Llama 3.1 8B',
    'gemma': 'Gemma 2 9B',
    'eurollm': 'EuroLLM 9B',
}

DATASETS = {
    'metricx-metricx-24-hybrid-xxl-v2p6': [
        'wmt14_csen',
        'wmt14_deen',
        'wmt14_ruen',
        'wmt14_fren',
        'wmt19_deen',
        'wmt19_fien',
        'wmt19_lten',
        'wmt19_ruen',
    ],
    'XComet-XCOMET-XXL': [
        'wmt14_csen',
        'wmt14_deen',
        'wmt14_ruen',
        'wmt14_fren',
        'wmt19_deen',
        'wmt19_fien',
        'wmt19_lten',
        'wmt19_ruen',
    ],
    'Comet-wmt22-comet-da': [
        'wmt14_csen',
        'wmt14_deen',
        'wmt14_ruen',
        'wmt14_fren',
        'wmt19_deen',
        'wmt19_fien',
        'wmt19_lten',
        'wmt19_ruen',
    ],
    'bleu_proper': [
        'wmt14_csen',
        'wmt14_deen',
        'wmt14_ruen',
        'wmt14_fren',
        'wmt19_deen',
        'wmt19_fien',
        'wmt19_lten',
        'wmt19_ruen',
    ],
}

METRICS = {
    'metricx-metricx-24-hybrid-xxl-v2p6': 'MetricX XXL',
    'XComet-XCOMET-XXL': 'XComet XXL',
    'Comet-wmt22-comet-da': 'Comet WMT22',
    # 'bleu_proper': 'BLEU',
}

pathlib.Path('tables').mkdir(parents=True, exist_ok=True)
pathlib.Path('charts').mkdir(parents=True, exist_ok=True)

def get_header(caption):
    return (
        "\\begin{table*}\n"
        "\\footnotesize\n"
        f"\caption{{{caption}}}\n"
        "\\begin{tabular}{lcccccccc}\n"
        "&\multicolumn{4}{c}{\\textbf{WMT14}}&\multicolumn{4}{c}{\\textbf{WMT19}}\\\\\n"
        "\cmidrule(lr){2-5}\n"
        "\cmidrule(lr){6-9}\n"
    )

def footer():
    return (
        "\midrule\n"
        "\end{tabular}\n"
        "\end{table*}\n"
    )

def colname(dataset):
    if '_' in dataset:
        dataset = dataset.split('_')[1]

    return dataset[:2].capitalize() + '-' + dataset[2:].capitalize()
def format_score_pair(raw, detr, threshold=0.00):
    arrow = "$\\uparrow$" if detr - raw > threshold else ""
    return f"{raw:.2f}", f"{detr:.2f}{arrow}"

def main(type_):
    caption = "Best raw and detrended PRR scores across all metrics and models."
    latex = "\\begin{table*}[ht]\n\\centering\n\\small\n"
    latex += f"\\caption{{{caption}}}\n"
    total_cols = max(len(v) for v in DATASETS.values())
    col_spec = "l" + "cc" * total_cols
    latex += f"\\begin{{tabular}}{{{col_spec}}}\n\\toprule\n"

    # Dataset header (generic, since all blocks will have same layout)
    dataset_line = "Model"
    subheader_line = " "
    example_metric = next(iter(METRICS))
    for d in DATASETS[example_metric]:
        name = colname(d)
        dataset_line += f" & \\multicolumn{{2}}{{c}}{{{name}}}"
        subheader_line += " & Raw & Detr"
    latex += dataset_line + " \\\\\n"
    latex += subheader_line + " \\\\\n\\midrule\n"

    for metric, metric_name in METRICS.items():
        datasets = DATASETS[metric]
        ue_methods = list(methods_dict.values())

        # Section header for metric
        n_cols = 1 + 2 * len(datasets)
        latex += f"\\multicolumn{{{n_cols}}}{{c}}{{\\textbf{{{metric_name}}}}} \\\\\n"

        for model_key, model_name in MODELS.items():
            ue_scores, _, _ = detrend_ue(
                datasets, model_key, "base", [metric], ue_methods, methods_dict, return_unprocessed=True
            )

            row = [model_name]
            for i in range(len(datasets)):
                best_raw = -float("inf")
                best_detr = -float("inf")

                for method in ue_methods:
                    raw = ue_scores[f"{method}_raw"][i]
                    detr = ue_scores[f"{method}_detr"][i]
                    best_raw = max(best_raw, raw)
                    best_detr = max(best_detr, detr)

                raw_fmt, detr_fmt = format_score_pair(best_raw, best_detr)
                row.extend([raw_fmt, detr_fmt])

            latex += " & ".join(row) + " \\\\\n"

        latex += "\\midrule\n"

    latex += "\\bottomrule\n\\end{tabular}\n\\end{table*}\n"

    with open("tables/final_rowwise_metric_blocks.tex", "w") as f:
        f.write(latex)



if __name__ == "__main__":
    type_='all'
    main(type_)

Some weights of the model checkpoint at microsoft/deberta-large-mnli were not used when initializing DebertaForSequenceClassification: ['config']
- This IS expected if you are initializing DebertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DebertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Stat calculators: [<lm_polygraph.stat_calculators.greedy_probs.GreedyProbsCalculator object at 0x7f41cce629e0>]
llama wmt19_ruen Below q ids: 1752


/home/maiya.goloburda/.conda/envs/detrend/lib/python3.10/site-packages/torch/cuda/__init__.py:734: UserWarning: Can't initialize NVML
  warnings.warn("Can't initialize NVML")
Some weights of the model checkpoint at microsoft/deberta-large-mnli were not used when initializing DebertaForSequenceClassification: ['config']
- This IS expected if you are initializing DebertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DebertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Stat calculators: [<lm_polygraph.stat_calculators.greedy_probs.GreedyProbsCalculator object at 0x7f426e6075e0>]


Some weights of the model checkpoint at microsoft/deberta-large-mnli were not used when initializing DebertaForSequenceClassification: ['config']
- This IS expected if you are initializing DebertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DebertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


...

/home/maiya.goloburda/.conda/envs/detrend/lib/python3.10/site-packages/torch/cuda/__init__.py:734: UserWarning: Can't initialize NVML
  warnings.warn("Can't initialize NVML")
Some weights of the model checkpoint at microsoft/deberta-large-mnli were not used when initializing DebertaForSequenceClassification: ['config']
- This IS expected if you are initializing DebertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DebertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Stat calculators: [<lm_polygraph.stat_calculators.greedy_probs.GreedyProbsCalculator object at 0x7f426e61a170>]


Some weights of the model checkpoint at microsoft/deberta-large-mnli were not used when initializing DebertaForSequenceClassification: ['config']
- This IS expected if you are initializing DebertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DebertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Error during conversion: ChunkedEncodingError(ProtocolError('Response ended prematurely'))


Stat calculators: [<lm_polygraph.stat_calculators.greedy_probs.GreedyProbsCalculator object at 0x7f41cce63910>]
eurollm wmt19_ruen Below q ids: 1762


Error during conversion: ChunkedEncodingError(ProtocolError('Response ended prematurely'))
